In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
df_meal

In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
len(speech_play.index)

In [ ]:
df_grouped_play

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

In [ ]:
speech_meal

In [ ]:
# remove label column from df_grouped_meal 
df_grouped_meal.reset_index()
df_grouped_meal.set_index('s_id', append=True, inplace=True)


In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
meal_data.columns

In [ ]:
meal_data = meal_data.drop(['label'], axis=1)
meal_data

In [ ]:
meal_data.columns

In [ ]:
meal_data = meal_data.dropna()

In [ ]:
df_grouped_play

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
play_data = play_data.dropna()

In [ ]:
play_data

## Classification with just one movement either cg or child

In [ ]:
X = df_grouped_meal[[
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max']]
y = df_grouped_meal['label']
gkf = KFold(n_splits=5)

results = []
best_models = {}
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01], 
        'mlp__solver': ['adam', 'lbfgs']
    })
]
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled  = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', n_jobs=-1)
        
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })
        
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
            best_models[name] = {
                'best_estimator': grid_search.best_estimator_,
                'best_params': grid_search.best_params_,
                'best_score': grid_search.best_score_
            }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').std() 

# Classification with just movement

In [ ]:
len(X)

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_meal[[
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max']]
y = df_grouped_meal['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').std()

In [ ]:
 results_df.loc[results_df['f1_score'].idxmax()][2]

# sanity check combine movement from play and meal

## Speech child + movement child

In [ ]:

# Custom transformer to apply PCA to a subset of features

# Pipeline with PCA applied to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)
    
X = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').std()

In [ ]:
results_df

## save the model

In [ ]:
results_df{'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 1}

In [ ]:
# check correlation of cg_movement_mean and child_movement_mean in meal_data
play_data[['cg_movement_mean', 'child_movement_mean']].corr()

# speech child play + meal movement

In [ ]:
df_grouped_meal.index, speech_play.index

In [ ]:
meal_mov_speech_play = pd.merge(df_grouped_meal[[
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max','label']], 
       speech_play['Proportion speech child'], on='s_id')

In [ ]:
len(meal_mov_speech_play)

In [ ]:

# Custom transformer to apply PCA to a subset of features

# Pipeline with PCA applied to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)
    
X = meal_mov_speech_play[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = meal_mov_speech_play['label']


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
best_models['svc_rbf']['best_estimator']

In [ ]:
result = permutation_importance(best_models['svc_rbf']['best_estimator'], X, y, n_repeats=30, random_state=0, scoring='accuracy')


In [ ]:
feature_importances_combined = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result['importances_mean']
})

feature_importances_combined = feature_importances_combined.sort_values(by='Importance', ascending=False)
# Plot the feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_combined)
plt.title('Feature Importances Play Speech + Meal Movement')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Movement feature differences per class 

In [ ]:
features_play = df_grouped_play[['cg_movement_max', 'cg_movement_min', 'cg_movement_mean', 'cg_movement_var',
 'child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min', 'label']]
features_meal = df_grouped_meal[['cg_movement_max', 'cg_movement_min', 'cg_movement_mean', 'cg_movement_var',
                                 'child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min', 'label']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a figure with subplots
features = ['child_movement_max', 'child_movement_min', 'child_movement_mean', 'child_movement_var']
label_column = 'label'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.boxplot(x=label_column, y=feature, data=df_grouped_play, ax=axes[i])
    axes[i].set_title(f'{feature} by {label_column}')
    axes[i].set_xlabel('Label')
    axes[i].set_ylabel(feature)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a figure with subplots
features = ['child_movement_max', 'child_movement_min', 'child_movement_mean', 'child_movement_var']
label_column = 'label'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.boxplot(x=label_column, y=feature, data=df_grouped_meal, ax=axes[i])
    axes[i].set_title(f'{feature} by {label_column}')
    axes[i].set_xlabel('Label')
    axes[i].set_ylabel(feature)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)
final_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a figure with subplots
features = ['child_movement_max', 'child_movement_min', 'child_movement_mean', 'child_movement_var']
label_column = 'label'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.boxplot(x=label_column, y=feature, data=final_df, ax=axes[i])
    axes[i].set_title(f'{feature} by {label_column}')
    axes[i].set_xlabel('Label')
    axes[i].set_ylabel(feature)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(x=final_df['label'], y=final_df['average_proximity'], data=final_df)

## Testing for differences between features in the two classes 

In [ ]:
# For the combined dataset
from scipy.stats import mannwhitneyu

# Example data
# Assuming final_df is your DataFrame and 'label' is the column with the labels
feature = 'cg_movement_max'
label_column = 'label'

# Separate the data into two groups based on the labels
group_0 = final_df[final_df[label_column] == 0][feature]
group_1 = final_df[final_df[label_column] == 1][feature]

# Perform the Mann-Whitney U test
stat, p_value = mannwhitneyu(group_0, group_1)

print(f'Mann-Whitney U test statistic: {stat}')
print(f'Mann-Whitney U test p-value: {p_value}')

# Interpretation
alpha = 0.05
if p_value < alpha:
    print('There is a significant difference between the two groups.')
else:
    print('There is no significant difference between the two groups.')

In [ ]:
# For the separate datasets
# For the combined dataset
from scipy.stats import mannwhitneyu

# Example data
# Assuming final_df is your DataFrame and 'label' is the column with the labels
feature = 'cg_movement_var'
label_column = 'label'

# Separate the data into two groups based on the labels
group_0 = df_grouped_play[df_grouped_play[label_column] == 0][feature]
group_1 = df_grouped_play[df_grouped_play[label_column] == 1][feature]

# Perform the Mann-Whitney U test
stat, p_value = mannwhitneyu(group_0, group_1)

print(f'Mann-Whitney U test statistic: {stat}')
print(f'Mann-Whitney U test p-value: {p_value}')

# Interpretation
alpha = 0.05
if p_value < alpha:
    print('There is a significant difference between the two groups.')
else:
    print('There is no significant difference between the two groups.')

# Combining speech child + movement features (with PCA)

In [ ]:
play_data = play_data.dropna()
play_data

In [ ]:
# Custom transformer to apply PCA to a subset of features

# Pipeline with PCA applied to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)
    

X = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:,0]


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
results_df

## take the best performing model outside of the pipeline and test it and then validate it on the other dataset

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
from sklearn.inspection import permutation_importance

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_meal = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y_meal = meal_data['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=0)
remaining_features = [feat for feat in X_meal.columns if feat not in subset_pca]
best_params = {'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 1}
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters
svc_rbf = SVC(C=best_params['svc_rbf__C'], gamma=best_params['svc_rbf__gamma'], kernel='rbf', probability=True)
# Create the final pipeline
pipeline_meal = Pipeline([
    ('preprocessor', preprocessor),
    ('svc_rbf', svc_rbf)
])

# Fit the pipeline
pipeline_meal.fit(X_meal, y_meal)

# Evaluate the model using cross-validation
scores = cross_val_score(pipeline_meal, X_meal, y_meal, scoring = 'accuracy', cv=cv)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {np.mean(scores)}')

# Make predictions on the dataset





In [ ]:
result = permutation_importance(pipeline_meal, X_meal, y_meal, n_repeats=30, random_state=0, scoring='accuracy')


In [ ]:
feature_importances_df = pd.DataFrame({
    'Feature': X_meal.columns,
    'Importance': result['importances_mean']
})

In [ ]:
feature_importances_meal = feature_importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
# Plot the feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_meal)
plt.title('Feature Importances Meal Data')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_play = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y_play = play_data['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=9)
remaining_features = [feat for feat in X_play.columns if feat not in subset_pca]
best_params = {'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 0.01}
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters
scv_rbf = SVC(C=best_params['svc_rbf__C'], gamma=best_params['svc_rbf__gamma'], kernel='rbf', probability=True)
# Create the final pipeline
pipeline_play = Pipeline([
    ('preprocessor', preprocessor),
    ('svc_rbf', svc_rbf)
])

# Fit the pipeline
pipeline_play.fit(X_play, y_play)

# Evaluate the model using cross-validation
scores = cross_val_score(pipeline_play, X_play, y_play, scoring = 'accuracy', cv=cv)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {np.mean(scores)}')

# Make predictions on the dataset





In [ ]:
result = permutation_importance(pipeline_play, X_play, y_play, n_repeats=30, random_state=0, scoring='accuracy')


In [ ]:
feature_importances_df = pd.DataFrame({
    'Feature': X_play.columns,
    'Importance': result['importances_mean']
})
features_importances_play = feature_importances_df.sort_values(by='Importance', ascending=False)


In [ ]:
# Plot the feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=features_importances_play)
plt.title('Feature Importances Play Data')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
meal_data_selected = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group','label']]
play_data_selected = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group', 'label']]

# Find the common indices
common_indices = meal_data_selected.index.intersection(play_data_selected.index)

# Filter the DataFrames to keep only the common indices
meal_data_filtered = meal_data_selected.loc[common_indices]
play_data_filtered = play_data_selected.loc[common_indices]

# Concatenate the filtered DataFrames
combined_data = pd.concat([meal_data_filtered, play_data_filtered])

In [ ]:
meal_data_filtered.iloc[:,:-3]

In [ ]:
## correlations between the two models 
# Get predictions from the LR model
meal_predictions =  pipeline_meal.predict_proba(meal_data_filtered.iloc[:,:-3])[:,1]

# Get predictions from the MLP model
play_predictions = pipeline_play.predict_proba(play_data_filtered.iloc[:,:-3])[:,1]

In [ ]:
accuracy_meal = pipeline_play.score(X_meal, y_meal)
print(f'Accuracy of model trained on play data evaluated on meal data: {accuracy_meal}')

In [ ]:
y_pred = pipeline_play.predict(X_meal)
f1 = f1_score(y_meal, y_pred, average='weighted')
precision = precision_score(y_meal, y_pred, average='weighted')
recall = recall_score(y_meal, y_pred, average='weighted')
print(f1, precision, recall)

In [ ]:
accuracy_play = pipeline_meal.score(X_play, y_play)
print(f'Accuracy of model trained on meal data evaluated on play data: {accuracy_play}')

In [ ]:
y_pred = pipeline_meal.predict(X_play)
f1 = f1_score(y_play, y_pred, average='weighted')
precision = precision_score(y_play, y_pred, average='weighted')
recall = recall_score(y_play, y_pred, average='weighted')
print(f1, precision, recall)

In [ ]:
combined_data = combined_data.dropna()
combined_labels = combined_data['label'].iloc[:,0]

# Combining the two models

In [ ]:
combined_data

## soft voting

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier


# Define the base models for voting
base_models = [
    ('play', pipeline_play),
    ('meal', pipeline_meal)
]

gkf = GroupKFold(n_splits=5)

# Create the voting classifier
voting_ensemble = VotingClassifier(estimators=base_models, voting='soft', weights = [0.4, 0.6])

# Evaluate the voting ensemble using cross-validation
f1_scores = cross_val_score(voting_ensemble, combined_data, combined_data.iloc[:,-1], cv=gkf, groups = combined_data.index, scoring='f1_macro', n_jobs=-1)
precision_scores = cross_val_score(voting_ensemble, combined_data, combined_data.iloc[:,-1], cv=gkf,groups = combined_data.index, scoring='precision_macro',  n_jobs=-1)
recall_scores = cross_val_score(voting_ensemble, combined_data, combined_data.iloc[:,-1], cv=gkf, groups = combined_data.index,scoring='recall_macro',  n_jobs=-1)
accuracy_scores = cross_val_score(voting_ensemble, combined_data, combined_data.iloc[:,-1], cv=gkf,groups = combined_data.index, scoring='accuracy', n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}") 

## stacking

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np


cv = KFold(n_splits=5, shuffle=True, random_state=42)
# Get cross-validated predictions for both models
meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict_proba')
play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict_proba')

# Align the predictions based on participants
# Assuming the index of X_meal and X_play are participant IDs
meal_predictions_df = pd.DataFrame(meal_predictions, columns=['meal_class_0', 'meal_class_1'], index=X_meal.index)
play_predictions_df = pd.DataFrame(play_predictions, columns=['play_class_0', 'play_class_1'], index=X_play.index)

# Combine the aligned predictions
combined_predictions = pd.concat([meal_predictions_df, play_predictions_df], axis=1).dropna()
combined_labels = pd.concat([y_meal, y_play], axis=1).dropna().iloc[:,0]

# Define the meta-model
meta_model = SVC(kernel='rbf', probability=True)


# Define the parameter grid for the meta-model
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 10, 100]
}

# Perform grid search for the meta-model
grid_search = GridSearchCV(meta_model, param_grid, cv=cv, scoring='accuracy')
grid_search.fit(combined_predictions, combined_labels)

# Evaluate the best meta-model using cross-validation
best_meta_model = grid_search.best_estimator_
cross_val_scores = cross_val_score(best_meta_model, combined_predictions, combined_labels, cv=cv, scoring='accuracy')

# Calculate cross-validated accuracy, F1 score, precision, and recall
meta_predictions = cross_val_predict(best_meta_model, combined_predictions, combined_labels, cv=cv)
accuracy = accuracy_score(combined_labels, meta_predictions)
f1 = f1_score(combined_labels, meta_predictions, average='weighted')
precision = precision_score(combined_labels, meta_predictions, average='weighted')
recall = recall_score(combined_labels, meta_predictions, average='weighted')

print(f'Best Meta-Model Parameters: {grid_search.best_params_}')
print(f'Cross-Validated Accuracy: {cross_val_scores.mean()}')
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# Looking at relationship between the two models

In [ ]:
## correlations between the two models 
# Get predictions from the LR model
meal_predictions =  pipeline_meal.predict_proba(meal_data_filtered.iloc[:,:-3])[:,1]

# Get predictions from the MLP model
play_predictions = pipeline_play.predict_proba(play_data_filtered.iloc[:,:-3])[:,1]



In [ ]:
# Compute the correlation between the predictions


In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
predictions_meal = cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:,:-3], combined_labels, cv=cv)
predictions_play = cross_val_predict(pipeline_play, play_data_filtered.iloc[:,:-3], combined_labels, cv=cv)

In [ ]:
meals = []
plays = []

for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    meals.append(cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:,:-3], combined_labels, cv=cv))
    plays.append(cross_val_predict(pipeline_play, play_data_filtered.iloc[:,:-3], combined_labels, cv=cv))

In [ ]:
from scipy.stats import pearsonr
correlations = []
for x, y in zip(meals, plays):
    correlation, p_value = pearsonr(x, y)
    correlations.append(correlation)

print(f"Correlation between LR and MLP model predictions: {np.mean(correlations)}")

In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa_avg = []
# Calculate Cohen's Kappa
for x, y in zip (meals, plays):
    kappa_score = cohen_kappa_score(x, y)
    kappa_avg.append(kappa_score)
print("Cohen's Kappa score:", np.mean(kappa_avg))


In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix
p_values = []
for x, y in zip (meals, plays):
    contingency_table = confusion_matrix(predictions_play, predictions_meal)
    result = mcnemar(contingency_table)
    p_values.append(result.pvalue)


print("McNemar's test p-value:", np.mean(p_values))

In [ ]:
agreements = []
for x,y in zip(meals, plays):
    agreement_rate = np.mean(x == y)
    agreements.append(agreement_rate)

print("Prediction Agreement Rate:", np.mean(agreements))

# Getting the accuracy for the cases where the two classifiers output the same prediction

In [ ]:
meal_data_filtered.iloc[:,-1]

In [ ]:
predictions_play

In [ ]:
same_predictions_indices = predictions_meal == predictions_play
filtered_meal_data = meal_data_filtered[same_predictions_indices]
filtered_play_data = play_data_filtered[same_predictions_indices]


true_labels_meal = meal_data_filtered.iloc[:, -1][same_predictions_indices]
true_labels_play = play_data_filtered.iloc[:, -1][same_predictions_indices]

# Calculate accuracy score
accuracy_meal = accuracy_score(true_labels_meal, predictions_meal[same_predictions_indices])
accuracy_play = accuracy_score(true_labels_play, predictions_play[same_predictions_indices])
# Calculate precision, recall, and f1 score
precision_meal = precision_score(true_labels_meal, predictions_meal[same_predictions_indices], average='weighted')
recall_meal = recall_score(true_labels_meal, predictions_meal[same_predictions_indices], average='weighted')
f1_meal = f1_score(true_labels_meal, predictions_meal[same_predictions_indices], average='weighted')

precision_play = precision_score(true_labels_play, predictions_play[same_predictions_indices], average='weighted')
recall_play = recall_score(true_labels_play, predictions_play[same_predictions_indices], average='weighted')
f1_play = f1_score(true_labels_play, predictions_play[same_predictions_indices], average='weighted')



print(f"Accuracy for same predictions: {accuracy_play}")
print(f"Precision for same predictions: {precision_play}")
print(f"Recall for same predictions: {recall_play}")
print(f"F1 Score for same predictions: {f1_play}")

In [ ]:
accuracies = []
precisions = []
recalls = []
f1_scores = []
lenghts = []
for x,y in zip(meals, plays):
    same_predictions_indices = x == y
    filtered_meal_data = meal_data_filtered[same_predictions_indices]
    filtered_play_data = play_data_filtered[same_predictions_indices]
    lenghts.append(len(filtered_meal_data))
    true_labels_meal = meal_data_filtered.iloc[:, -1][same_predictions_indices]
    true_labels_play = play_data_filtered.iloc[:, -1][same_predictions_indices]
    # Calculate accuracy score
    accuracy_meal = accuracy_score(true_labels_meal, x[same_predictions_indices])
    # Calculate precision, recall, and f1 score
    precision_meal = precision_score(true_labels_meal, x[same_predictions_indices], average='weighted')
    recall_meal = recall_score(true_labels_meal, x[same_predictions_indices], average='weighted')
    f1_meal = f1_score(true_labels_meal, x[same_predictions_indices], average='weighted')
    accuracies.append(accuracy_meal)
    precisions.append(precision_meal)
    recalls.append(recall_meal)
    f1_scores.append(f1_meal)


print(f"Accuracy for same predictions: {np.mean(accuracies)}")
print(f"Precision for same predictions: {np.mean(precisions)}")
print(f"Recall for same predictions: {np.mean(recalls)}")
print(f"F1 Score for same predictions: {np.mean(f1_scores)}")
print(f"Number of same predictions: {np.mean(lenghts)}")

In [ ]:
len(filtered_play_data)

In [ ]:
33.5/49

In [ ]:
combined_data

In [ ]:
combined_data.iloc[:,:-3]

In [ ]:
X

In [ ]:
weights = [0.6, 0.4]  # Adjust these weights based on model performance

# Initialize lists to store the evaluation metrics
f1_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []
gkf = GroupKFold(n_splits=5)
X = combined_data.iloc[:,:-3]
y = combined_data['label'].iloc[:,0]
groups = combined_data.index
# Perform manual cross-validation
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the base models
    for name, model in base_models:
        model.fit(X_train, y_train)
    
    # Get the predicted probabilities from each base model
    meal_probs = pipeline_meal.predict_proba(X_test)[:, 1]
    play_probs = pipeline_play.predict_proba(X_test)[:, 1]
    
    # Combine the predicted probabilities using weighted averaging
    combined_probs = (weights[0] * meal_probs) + (weights[1] * play_probs)
    
    # Convert combined probabilities to class predictions
    combined_preds = np.where(combined_probs > 0.5, 1, 0)
    
    # Calculate evaluation metrics
    f1_scores.append(f1_score(y_test, combined_preds, average='macro'))
    precision_scores.append(precision_score(y_test, combined_preds, average='macro'))
    recall_scores.append(recall_score(y_test, combined_preds, average='macro'))
    accuracy_scores.append(accuracy_score(y_test, combined_preds))

# Print the evaluation results
print(f"F1 Score: {np.mean(f1_scores)}")
print(f"Precision: {np.mean(precision_scores)}")
print(f"Recall: {np.mean(recall_scores)}")
print(f"Accuracy: {np.mean(accuracy_scores)}")